In [ ]:
#@title 📌 Video Downloader Instructions
#@markdown 1. Create a file named `urls.txt` in this folder.<br>
#@markdown 2. Paste all the video URLs into `urls.txt`, one URL per line.<br>
#@markdown 3. Run this cell to download all videos with a progress bar and create a ZIP archive.<br>
#@markdown 4. Click the button below to download the ZIP manually.

# Folder name for downloads
save_folder_name = "downloads"  #@param ["downloads", "videos", "output"] {allow-input: true}

import requests
from pathlib import Path
from urllib.parse import urlparse, parse_qs
import shutil
from google.colab import files
from tqdm.notebook import tqdm

save_folder = Path(save_folder_name)
save_folder.mkdir(exist_ok=True)

# Map common mime_type values to extensions
mime_map = {
    "video_mp4": ".mp4",
    "video_webm": ".webm",
    "image_jpeg": ".jpg",
    "image_png": ".png"
}

# Read URLs from urls.txt
url_file = "urls.txt"
try:
    with open(url_file, 'r') as f:
        urls = [line.strip() for line in f if line.strip()]
except FileNotFoundError:
    print("❌ urls.txt not found! Please create the file and paste URLs in it.")
    urls = []

# Download videos with progress bar
for idx, url in enumerate(tqdm(urls, desc="Downloading videos", unit="file")):
    try:
        # Detect mime_type from URL query
        parsed_url = urlparse(url)
        qs = parse_qs(parsed_url.query)
        mime_type = qs.get("mime_type", [""])[0]
        extension = mime_map.get(mime_type, ".bin")

        filename = save_folder / f"file_{idx+1}{extension}"

        resp = requests.get(url, stream=True)
        resp.raise_for_status()

        with open(filename, 'wb') as out_file:
            for chunk in resp.iter_content(chunk_size=1024*1024):
                if chunk:
                    out_file.write(chunk)

    except Exception as e:
        print(f"❌ Failed to download {url}: {e}")

# Create ZIP archive of the folder
zip_name = save_folder_name + ".zip"
shutil.make_archive(save_folder_name, 'zip', save_folder)
print(f"\n🎉 All files zipped into: {zip_name}")

# Download button
files.download(zip_name)
